In [1]:
# Creates the folder kemenyGPU
!git clone https://github.com/noeliarico/kemenyGPU

Cloning into 'kemenyGPU'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 49 (delta 23), reused 27 (delta 7), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [33]:
from numba import cuda
import numpy as np
import math
import time
import logging
logging.getLogger().setLevel(logging.INFO)

In [34]:
execfile('kemenyGPU/v1.py')
execfile('kemenyGPU/v2.py')
execfile('kemenyGPU/v2d.py')
execfile('kemenyGPU/v3.py')
execfile('kemenyGPU/oms.py')

In [121]:
om = om12
n = 12
m = 10

# Execution time `v1`

- `n = 11`: [0.12]
- `n = 12`: [0.82]
- `n = 13`: [12.81]
- `n = 14`: [214] 

In [125]:
stride = 4000000

# create an array of dimension stride x n (initially empty)
# to store the rankings and factorial representations
d_factoradic = cuda.device_array((stride,n), dtype=np.uint8)

# to store the boolean and factorial representations
d_alternatives = cuda.device_array((stride,n), dtype=np.bool_)

threadsperblock = 256
blockspergrid = math.ceil(stride / threadsperblock)
logging.info("threadsperblock = {}, blockspergrid = {}, total = {}".format(threadsperblock, blockspergrid, threadsperblock*blockspergrid))

# array of one position to store the best distance
best_dist = np.array([(n*n-n)/2]) # initiallize with upper bound
# move the value to the gpu
d_best_dist = cuda.to_device(best_dist)
# move the value to the gpu
best_ranking = np.array([0], dtype=np.uint64) 
d_best_ranking = cuda.to_device(best_ranking)
# single = False
total = math.factorial(n)

start = time.time()
v1[blockspergrid, threadsperblock](d_factoradic, d_alternatives, om, stride, total, d_best_dist, d_best_ranking)
d_best_dist.copy_to_host(best_dist)
d_best_ranking.copy_to_host(best_ranking)
end = time.time()


logging.info("Execution time: {}".format(end-start))

factoradic = d_factoradic.copy_to_host()
print(factoradic[:26, :])
logging.info(best_dist)
logging.info(best_ranking)

INFO:root:threadsperblock = 256, blockspergrid = 15625, total = 4000000
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 48000000 bytes
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 48000000 bytes
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 8 bytes
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 8 bytes
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 1152 bytes
INFO:root:Execution time: 0.9391634464263916
INFO:root:[66.]
INFO:root:[0]


[[11 10  1  7  5  4  0  8  3  6  2  9]
 [11 10  1  7  5  4  0  8  3  6  9  2]
 [11 10  1  7  5  4  0  8  3  9  2  6]
 [11 10  1  7  5  4  0  8  3  9  6  2]
 [11 10  1  7  5  4  0  8  6  2  3  9]
 [11 10  1  7  5  4  0  8  6  2  9  3]
 [11 10  1  7  5  4  0  8  6  3  2  9]
 [11 10  1  7  5  4  0  8  6  3  9  2]
 [11 10  1  7  5  4  0  8  6  9  2  3]
 [11 10  1  7  5  4  0  8  6  9  3  2]
 [11 10  1  7  5  4  0  8  9  2  3  6]
 [11 10  1  7  5  4  0  8  9  2  6  3]
 [11 10  1  7  5  4  0  8  9  3  2  6]
 [11 10  1  7  5  4  0  8  9  3  6  2]
 [11 10  1  7  5  4  0  8  9  6  2  3]
 [11 10  1  7  5  4  0  8  9  6  3  2]
 [11 10  1  7  5  4  0  9  2  3  6  8]
 [11 10  1  7  5  4  0  9  2  3  8  6]
 [11 10  1  7  5  4  0  9  2  6  3  8]
 [11 10  1  7  5  4  0  9  2  6  8  3]
 [11 10  1  7  5  4  0  9  2  8  3  6]
 [11 10  1  7  5  4  0  9  2  8  6  3]
 [11 10  1  7  5  4  0  9  3  2  6  8]
 [11 10  1  7  5  4  0  9  3  2  8  6]
 [11 10  1  7  5  4  0  9  3  6  2  8]
 [11 10  1  7  5  4  0  9

# Execution time `v2`

In this case each thread keeps the local minimum. This reduces the execution time because reduces the number of atomical accesses

- `n = 11`: [0.11]
- `n = 12`: [0.91]
- `n = 13`: [12.81]
- `n = 14`: [213] 

In [124]:
# create an array of dimension factorial(n) x n (initially empty)
# to store in each i-th row the factorial representation of i
stride = 4000000
# to store the rankings and factorial representations
# factoradic = np.zeros((stride,n), dtype=np.int64)
# d_factoradic = cuda.to_device(factoradic)
d_factoradic = cuda.device_array((stride,n), dtype=np.uint8)
# to store the boolean and factorial representations
# alternatives = np.zeros((stride,n), dtype=np.bool_)
# d_alternatives = cuda.to_device(alternatives)
d_alternatives = cuda.device_array((stride,n), dtype=np.bool_)


threadsperblock = 512
blockspergrid = math.ceil(stride / threadsperblock)
logging.info("threadsperblock = {}, blockspergrid = {}, total = {}".format(threadsperblock, blockspergrid, threadsperblock*blockspergrid))
best_dist = np.array([10000]) 
d_best_dist = cuda.to_device(best_dist)
best_ranking = np.array([0], dtype=np.uint64) 
d_best_ranking = cuda.to_device(best_ranking)
single = False

total = math.factorial(n)

start = time.time()
v2[blockspergrid, threadsperblock](d_factoradic, d_alternatives, om, stride, total, d_best_dist, d_best_ranking)


d_best_dist.copy_to_host(best_dist)
d_best_ranking.copy_to_host(best_ranking)

factoradic = d_factoradic.copy_to_host()
print(factoradic[:26, :])

end = time.time()
logging.info("Execution time: {}".format(end-start))
logging.info(best_dist)
logging.info(best_ranking)

INFO:root:threadsperblock = 512, blockspergrid = 7813, total = 4000256
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 4800000 bytes
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 8 bytes
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 8 bytes
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 1152 bytes
INFO:root:Execution time: 0.9731216430664062
INFO:root:[275]
INFO:root:[517423]


[[11 10  1  7  5  4  0  8  3  6  2  9]
 [11 10  1  7  5  4  0  8  3  6  9  2]
 [11 10  1  7  5  4  0  8  3  9  2  6]
 [11 10  1  7  5  4  0  8  3  9  6  2]
 [11 10  1  7  5  4  0  8  6  2  3  9]
 [11 10  1  7  5  4  0  8  6  2  9  3]
 [11 10  1  7  5  4  0  8  6  3  2  9]
 [11 10  1  7  5  4  0  8  6  3  9  2]
 [11 10  1  7  5  4  0  8  6  9  2  3]
 [11 10  1  7  5  4  0  8  6  9  3  2]
 [11 10  1  7  5  4  0  8  9  2  3  6]
 [11 10  1  7  5  4  0  8  9  2  6  3]
 [11 10  1  7  5  4  0  8  9  3  2  6]
 [11 10  1  7  5  4  0  8  9  3  6  2]
 [11 10  1  7  5  4  0  8  9  6  2  3]
 [11 10  1  7  5  4  0  8  9  6  3  2]
 [11 10  1  7  5  4  0  9  2  3  6  8]
 [11 10  1  7  5  4  0  9  2  3  8  6]
 [11 10  1  7  5  4  0  9  2  6  3  8]
 [11 10  1  7  5  4  0  9  2  6  8  3]
 [11 10  1  7  5  4  0  9  2  8  3  6]
 [11 10  1  7  5  4  0  9  2  8  6  3]
 [11 10  1  7  5  4  0  9  3  2  6  8]
 [11 10  1  7  5  4  0  9  3  2  8  6]
 [11 10  1  7  5  4  0  9  3  6  2  8]
 [11 10  1  7  5  4  0  9

# Execution time `v3`

Se devuelve la primera columna de la estructura que se usa para los factoriales y luego en la cpu se reduce el mínimo

In [123]:
# create an array of dimension factorial(n) x n (initially empty)
# to store in each i-th row the factorial representation of i
stride = 400000 # con un 0 menos no funciona porque salen mal las permutaciones 

# to store the rankings and factorial representations
# factoradic = np.zeros((stride,n), dtype=np.int64)
# d_factoradic = cuda.to_device(factoradic)
d_factoradic = cuda.device_array((stride,n), dtype=np.uint8)
# to store the boolean and factorial representations
# alternatives = np.zeros((stride,n), dtype=np.bool_)
# d_alternatives = cuda.to_device(alternatives)
d_alternatives = cuda.device_array((stride,n), dtype=np.bool_)


threadsperblock = 512
blockspergrid = math.ceil(stride / threadsperblock)
logging.info("threadsperblock = {}, blockspergrid = {}, total = {}".format(threadsperblock, blockspergrid, threadsperblock*blockspergrid))

total = math.factorial(n)

start = time.time()
v3[blockspergrid, threadsperblock](d_factoradic, d_alternatives, om, stride, total)

end = time.time()

#factoradic = d_factoradic.copy_to_host()

col = d_factoradic[:, 0]
dists = col.copy_to_host()
print(np.min(dists))


#print(factoradic[:26,:])
#print(dists[:total])

#end = time.time()


logging.info("Execution time: {}".format(end-start))
#print(dists[:26])

INFO:root:threadsperblock = 512, blockspergrid = 782, total = 400384
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 4800000 bytes
INFO:numba.cuda.cudadrv.driver:add pending dealloc: cuMemFree_v2 1152 bytes
INFO:root:Execution time: 0.937018632888794


19


In [90]:
import sys
np.set_printoptions(threshold=sys.maxsize)